# Clear Sky Composite of Goes-16 Images

## Step 1: Setting work environment

Importing libraries

In [167]:
import calendar, datetime, os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

Input/output path and locations/period of interest

In [118]:
# Input: path where the files with .nc extension are located
path_input = " "

# Output 1: path where the cropped .nc files will be saved
path_output_1 = " "

# Output 2: path where the composite files with .nc extensions will be saved
path_output_2 =  " "

# Output 3: path where the clear sky images will be saved
path_output_3 = " "

# Year and months of interest
# IMPORTANT: months must be in the string formar, in the 01-12 range
year = 2019
months = ['01', '04', '07', '10']

# Locations: list with names, latitudes and longitudes of the areas of interest
locations = [['brb', -15.60083, -47.71306],
             ['cpa', -22.6896, -45.0062],
             ['ptr', -9.0689, -40.3197],
             ['sms', -29.4428, -53.8231]]

## Part 2: cropping NetCDF files

In [81]:
# Setting file paths where the .nc files are located
directory = os.fsencode(path_input)
os.chdir(path_input)

# Looping through the selected months
for month in months:
    
    # Setting start and end date in julian day format
    day_start = calendar.monthrange(year, int(month))[0]
    day_end = calendar.monthrange(year, int(month))[1]

    julian_start = datetime.datetime.strptime(month + '.' + str(day_start), '%m.%d').timetuple().tm_yday
    julian_end = datetime.datetime.strptime(month + '.' + str(day_end), '%m.%d').timetuple().tm_yday

    # Creating list with range of julian days which will be further used to locate the respective files
    days = []

    for day in range(julian_start, julian_end + 1):
        if day < 10:
            days.append('00' + str(day))
        elif day < 100:
            days.append('0' + str(day))
        else:
            days.append(str(day))
    
    # Looping through locations
    for location in locations:
        
        # Looping through the files 
        for file in os.listdir(directory):
            
            file = file.decode('utf-8')
            
            # Matching the file with the desired month
            if file[-13:-10] in days:
                ds = xr.open_dataset(file)

                # Extracting the date from the file
                date = str(ds.attrs['date_created'])

                # Selecting the variables
                lon = ds['lon'].values
                lat = ds['lat'].values

                # Setting the value of k, this is the size of our box around the location
                k = 2.0

                # Setting the latitude and latitude
                lat_station = location[1]
                lon_station = location[2]
                lat_list = list(lat)
                lon_list = list(lon)

                # Finding the closest point in the dataset that matches our location latitude and longitude
                lat_start = lat_list.index(lat_list[min(range(len(lat_list)), key = lambda i: abs(lat_list[i]-(lat_station - k)))])
                lat_end = lat_list.index(lat_list[min(range(len(lat_list)), key = lambda i: abs(lat_list[i]-(lat_station + k)))])
                lon_start = lon_list.index(lon_list[min(range(len(lon_list)), key = lambda i: abs(lon_list[i]-(lon_station - k)))])
                lon_end = lon_list.index(lon_list[min(range(len(lon_list)), key = lambda i: abs(lon_list[i]-(lon_station + k)))])

                # Setting the new .nc file with our specified dimensions
                data = ds['CMI'][lat_start:lat_end, lon_start:lon_end]

                # Naming our file with the date and time
                file = location[0] + '_' + date[0:4] + '_' + date[5:7] + '_' + date[8:10] + '_' + date[11:13] + date[14:16]

                # Saving the cropped NetCDF file
                data.to_netcdf(path=(path_output_1 + f'\{file}.nc'))
                ds.close()

## Part 3: creating composite image

In [150]:
# Changing directory
directory = os.fsencode(path_output_1)
os.chdir(path_output_1)

# Looping through locations
for location in locations:
    
    # Looping through months
    for month in months:
        
        # Creating a list to store the different times of each image
        times = []

        # Looping through files in the folder
        for file in os.listdir(directory):

            file = file.decode('utf-8')

            # Matching the file name with the respective location and month
            if file[0:3] == location[0] and file[9:11] == month:

                if file[-7:-3] not in times:
                    times.append(file[-7:-3])

            #Looping through the times in the list
            for time in times:
                
                # Creating dummy array to kickstart the composite file
                ds1 = np.random.randint(1,5, size=(550,550))
                
                # Looping through the files in the folder
                for file in os.listdir(directory):
                    
                        # Opening the respective file that matches the time
                        if file.decode('utf-8')[-7:-3] == time:
                            
                            # Opening the dataset
                            ds2 = xr.open_dataset(file.decode('utf-8'))
                            
                            # Vectorizing dataset
                            ds1_ = np.reshape(ds1, (1, 550*550))
                            ds2_ = np.reshape(ds2.CMI.values, (1, 550*550))
                            
                            # Getting minimum values between both datasets
                            ds3 = np.reshape(np.minimum(ds1_,ds2_), (550,550))
                            ds1 = ds3

                # Creating new NetCDF file with minimum values           
                ds = xr.Dataset({"CMI": (("lat", "lon"), ds3)}, coords={"lat": ds2.lat.values,"lon": ds2.lon.values})
                
                # Setting file name and saving
                file = location[0]+ '_' + month + '_'+ time
                ds.to_netcdf(path=(path_output_2 + f'\{file}.nc'))
                
                # Resetting dummy dataset
                ds1 = np.random.randint(1,5, size=(550,550))

## Part 4: plotting the images

In [166]:
# Changing directory
directory = os.fsencode(path_output_2)
os.chdir(path_output_2)

# Looping through locations
for location in locations:
    
    # Looping through files in folder
    for file in os.listdir(directory):
        
        # Opening dataset
        file = file.decode('utf-8')
        ds = xr.open_dataset(file)

        # Plotting the image
        fig = plt.figure(figsize=(10,10))
        ax = plt.axes(projection=ccrs.PlateCarree())
        mapa = ds['CMI'].plot(ax=ax,transform=ccrs.PlateCarree(), x='lon', y='lat', add_colorbar=False, cmap='gray', vmax=0.8, vmin=0)
        cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
        plt.colorbar(mapa, cax=cax).remove()
        gl = ax.grid(True)

        # Saving image
        file = file[0:-3] + '.png'
        plt.savefig(path_output_3 + f'\{file}')
        plt.close()